## 1. ADD Data sets

In [ ]:
np.random.choice([[nm, ransize], False])

In [1]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0_blur import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '1' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', to = 'e')

#track1 = data_loader(pathset)
#track1_generator = DataGenerator(track1.eval, track1.labels, **params_no_shuffle)

#################################################
### get_ResMax
mname = 'ResMax_ADD_'
#################################################

### None

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        
### Mixup

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False # np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
#    ru = np.random.uniform(.1, .9)
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

   
### HP         

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
#    ru = np.random.uniform(.1, .9)
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)




################################
mname = "LCNN_ADD_"
################################

## None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        
        
### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        
### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)



###############################################
mname = "BCResMax_ADD_"
###############################################

### None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        

###############################################
mname = "DDWSseq_ADD_"
###############################################

### None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': add2022,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': add2022
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_ADD/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        

cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1692/1692 [==============================] - 46s 24ms/step - loss: 0.1985 - auc: 0.9786
Epoch 2/70
1692/1692 [==============================] - 41s 24ms/step - loss: 0.0204 - auc: 0.9996
Epoch 3/70
1692/1692 [==============================] - 41s 24ms/step - loss: 0.0058 - auc: 0.9999
Epoch 4/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0172 - auc: 0.9997
Epoch 5/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0013 - auc: 1.0000
Epoch 6/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0367 - auc: 0.9987
Epoch 7/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0055 - auc: 0.9998
Epoch 8/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0014 - auc: 1.0000
Epoch 9/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0030 - auc: 0.9998
Epoch 10/70
1692/1692 [==============================] - 42s 25ms/step - loss: 0.0044 - auc: 0.9998
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 9.610579325719515e-05


...

Epoch 1/70
1692/1692 [==============================] - 75s 40ms/step - loss: 1.2935 - auc: 0.8489
Epoch 2/70
1692/1692 [==============================] - 67s 40ms/step - loss: 0.5106 - auc: 0.9453
Epoch 3/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.4184 - auc: 0.9621
Epoch 4/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.3863 - auc: 0.9666
Epoch 5/70
1692/1692 [==============================] - 67s 40ms/step - loss: 0.3754 - auc: 0.9681
Epoch 6/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.3637 - auc: 0.9699
Epoch 7/70
1692/1692 [==============================] - 67s 40ms/step - loss: 0.3435 - auc: 0.9748
Epoch 8/70
1692/1692 [==============================] - 67s 40ms/step - loss: 0.3419 - auc: 0.9742
Epoch 9/70
1692/1692 [==============================] - 68s 40ms/step - loss: 0.3456 - auc: 0.9735
Epoch 10/70
1692/1692 [==============================] - 67s 40ms/step - loss: 0.3278 - auc: 0.9741
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.00042816443133840323


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.14334637964774954
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': 0.7, 'data_dir': '/home/ubuntu/data/ADD/', 'lowpass': [[0.5, 0.5], [7, 8, 9, 10, 11, 12]], 'highpass': [[0.5, 0.5], [80, 81, 82, 83, 84, 85, 86, 87]], 'ranfilter2': [2, [8, 9, 10, 11, 12]], 'model': 'DDWSseq_ADD_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0004', 'eer_eval': '0.1433', 'saved_model': 'saved_models/model_name!DDWSseq_ADD_!dropout_rate!0.5!human_weight!5.0!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!0.7!lowpass!--!highpass!--!ranfilter2!--!0.0004.hdf5', 'tnow': '2022-05-31 14:58:03.265752'}


## 2. ASV- LA Data sets

In [ ]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0_blur import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
#     except RuntimeError as e:
#         print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '2' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', pl_pick = 'l',  to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', pl_pick = 'l', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', pl_pick = 'l', to = 'e')

#track1 = data_loader(pathset)
#track1_generator = DataGenerator(track1.eval, track1.labels, **params_no_shuffle)

#################################################
### get_ResMax
mname = 'ResMax_LA_'
#################################################

### None

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        
        
### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'  
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP        
 
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
#    ru = np.random.uniform(.1, .9)
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

      

### HP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
#    ru = np.random.uniform(.1, .9)
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 # np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)




################################
mname = "LCNN_LA_"
################################

## None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        

### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        


##################################################
mname = "BCResMax_LA_"
##################################################

### None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


##################################################
mname = "DDWSseq_LA_"
##################################################

### None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 43s 26ms/step - loss: 0.4302 - auc: 0.9214
Epoch 2/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0745 - auc: 0.9978
Epoch 3/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0474 - auc: 0.9991
Epoch 4/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0361 - auc: 0.9994
Epoch 5/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0440 - auc: 0.9990
Epoch 6/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0218 - auc: 0.9997
Epoch 7/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0193 - auc: 0.9995
Epoch 8/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0165 - auc: 0.9996
Epoch 9/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0157 - auc: 0.9995
Epoch 10/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0122 - auc: 0.9997
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.007122201848719261


...

EER: 0.006897825614649971


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.03553930747626956
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': 0.7, 'data_dir': '/home/ubuntu/data/asv2019/', 'lowpass': [[0.5, 0.5], [7, 8, 9, 10, 11, 12]], 'highpass': False, 'ranfilter2': False, 'model': 'ResMax_LA_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0068', 'eer_eval': '0.0355', 'saved_model': 'saved_models/model_name!ResMax_LA_!dropout_rate!0.5!human_weight!5.0!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!0.7!lowpass!--!highpass!--!ranfilter2!--!0.0068.hdf5', 'tnow': '2022-05-31 21:30:30.089205'}
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 44s 26ms/step - loss: 0.6730 - auc: 0.8747
Epoch 2/70
1586/1586 [==============================] - 42s 26ms/step - loss: 0.3887 - auc: 0.9588
Epoch 3/70
1586/1586 [==============================] - 42s 27ms/step - loss: 0.3599 - auc: 0.9640
Epoch 4/70
1586/1586 [==============================] - 42s 27ms/step - loss: 0.3312 - auc: 0.9658
Epoch 5/70
1586/1586 [==============================] - 42s 27ms/step - loss: 0.3367 - auc: 0.9668
Epoch 6/70
1586/1586 [==============================] - 42s 27ms/step - loss: 0.3143 - auc: 0.9704
Epoch 7/70
1586/1586 [==============================] - 42s 26ms/step - loss: 0.3259 - auc: 0.9704
Epoch 8/70
1586/1586 [==============================] - 42s 26ms/step - loss: 0.3020 - auc: 0.9722
Epoch 9/70
1586/1586 [==============================] - 42s 27ms/step - loss: 0.3035 - auc: 0.9707
Epoch 10/70
1586/1586 [==============================] - 42s 27ms/step - loss: 0.3028 - auc: 0.9711
Epoch 11/

In [1]:
!nvidia-smi

Tue May 31 22:18:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   39C    P0    38W / 300W |      3MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [ ]:
## 렉걸렸음(22/5/31, 다시 돌리기)
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0_blur import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '2' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', pl_pick = 'l',  to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', pl_pick = 'l', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', pl_pick = 'l', to = 'e')

#track1 = data_loader(pathset)
#track1_generator = DataGenerator(track1.eval, track1.labels, **params_no_shuffle)

#################################################
### get_ResMax
mname = 'ResMax_LA_'
#################################################

### None

# for i in range(3) :
#     sr = 16000
#     sec = 9.0
#     batch_size = 16
#     feature = "cqt"
#     #hop_length = 128
#     #win_length = 512
#     #n_bins = 80
#     filter_scale = 1
#     n_bins = 100
#     fmin = 5
#     epoch = 70
#     beta_param = False
#     dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#     human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#     win_size = np.random.choice([10,15,20])
#     if win_size == 10 :
#         lpsize = [8,9,10,11,12]
#         hpsize = [88,89,90,91,92]
#         ransize = [8,9,10,11,12]
#     elif win_size == 15 :
#         lpsize = [13,14,15,16,17]
#         hpsize = [83,84,85,86,87]
#         ransize = [13,14,15,16,17]
#     else :
#         lpsize = [18,19,20,21,22]
#         hpsize = [78,79,80,81,82]
#         ransize = [18,19,20,21,22]
#     ru = 0.5#np.random.uniform(.1, .9)
#     uv = [ru, 1-ru]
#     lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
#     highpass = False #np.random.choice([[uv, hpsize], False])
#     nm = np.random.choice([2,3,4])
#     ranfilter2 = False #np.random.choice([[nm, ransize], False])
#     tmp_string = "tmp"
#     params = {'sr': sr,
#           'batch_size': batch_size,
#           'feature': feature,
#           'n_classes': 2,
#           'sec': sec,
#           'filter_scale': filter_scale,
#           'fmin' : fmin,
#           'n_bins': int(n_bins),
#           'tofile': tmp_string,
#           'shuffle': True,
#           'beta_param': beta_param,
#           'data_dir': asv2019,
#           'lowpass': lowpass,
#           'highpass': highpass,
#           #          'ranfilter' : [10,11,12,13,14,15]
#           'ranfilter2' : ranfilter2 
#           #           'dropblock' : [30, 100]
#           #'device' : device
#     }
#     params_no_shuffle = {'sr': sr,
#                      'batch_size': batch_size,
#                      'feature': feature,
#                      'n_classes': 2,
#                      'sec': sec,
#                      'filter_scale': filter_scale,
#                      'fmin' : fmin,
#                      'n_bins': int(n_bins),
#                      'tofile': tmp_string,
#                      'shuffle': False,
#                     'data_dir': asv2019
#                      #'device': device
#     }
#     training_generator = DataGenerator(dl.train, dl.labels, **params)
#     validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#     eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#     input_shape = training_generator.get_input_shape()
#     model = get_ResMax(input_shape)
# #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#     EPOCHS = epoch
#     lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#     class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#     history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                               epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#     eer_val = evalEER(validation_generator, model)
#     eer_eval = evalEER(eval_generator, model)
#     endtxt1 = str(eer_val)[:6] + '.hdf5'
#     savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
#     savefnm = 'saved_models/' + savefnm
#     model.save(savefnm)
#     endtxt1 = str(eer_val)[:6] + '.npy'
#     fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
#     fnm = 'saved_results/' + fnm
#     sc1 = model.predict(eval_generator)
#     np.save(fnm, sc1)
#     params_save = params
#     params_save['model'] = mname
#     params_save['human_weight'] = str(human_weight)
#     params_save['dropout_rate'] = str(dropout_rate)
#     params_save['eer_val'] = str(eer_val)[:6]
#     params_save['eer_eval'] = str(eer_eval)[:6]
#     params_save['saved_model'] = savefnm
#     tnow = datetime.datetime.now()
#     params_save['tnow'] = str(tnow)
#     print(params_save)
#     fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
#     with open(fnm, 'wb') as f:
#         pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        
### Mixup

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False # np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': False, # beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP        
 
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
#    ru = np.random.uniform(.1, .9)
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

      
### HP        
 
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
#    ru = np.random.uniform(.1, .9)
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + RP

for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 # np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)




################################
mname = "LCNN_LA_"
################################

## None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        

### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_LCNN(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        


##################################################
mname = "BCResMax_LA_"
##################################################

### None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_BCResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


##################################################
mname = "DDWSseq_LA_"
##################################################

### None
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = False
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### LP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP         
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 47s 26ms/step - loss: 0.3654 - auc: 0.9383
Epoch 2/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0883 - auc: 0.9978
Epoch 3/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0615 - auc: 0.9985
Epoch 4/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0542 - auc: 0.9989
Epoch 5/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0330 - auc: 0.9994
Epoch 6/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0243 - auc: 0.9995
Epoch 7/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0194 - auc: 0.9997
Epoch 8/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0205 - auc: 0.9994
Epoch 9/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0229 - auc: 0.9994
Epoch 10/70
1586/1586 [==============================] - 41s 26ms/step - loss: 0.0110 - auc: 0.9998
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.008612947685184016


...

EER: 0.006611842781932795


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.028564705498229793
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': 0.7, 'data_dir': '/home/ubuntu/data/asv2019/', 'lowpass': False, 'highpass': [[0.5, 0.5], [80, 81, 82, 83, 84, 85, 86, 87]], 'ranfilter2': False, 'model': 'DDWSseq_LA_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0066', 'eer_eval': '0.0285', 'saved_model': 'saved_models/model_name!DDWSseq_LA_!dropout_rate!0.5!human_weight!5.0!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!0.7!lowpass!--!highpass!--!ranfilter2!--!0.0066.hdf5', 'tnow': '2022-06-03 22:50:33.996107'}
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 66s 39ms/step - loss: 1.2885 - auc: 0.7913
Epoch 2/70
1586/1586 [==============================] - 61s 39ms/step - loss: 0.6659 - auc: 0.9201
Epoch 3/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.5011 - auc: 0.9442
Epoch 4/70
1586/1586 [==============================] - 61s 39ms/step - loss: 0.4510 - auc: 0.9542
Epoch 5/70
1586/1586 [==============================] - 61s 39ms/step - loss: 0.4198 - auc: 0.9599
Epoch 6/70
1586/1586 [==============================] - 61s 39ms/step - loss: 0.4026 - auc: 0.9603
Epoch 7/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.3803 - auc: 0.9615
Epoch 8/70
1586/1586 [==============================] - 61s 39ms/step - loss: 0.3677 - auc: 0.9631
Epoch 9/70
1586/1586 [==============================] - 61s 39ms/step - loss: 0.3740 - auc: 0.9663
Epoch 10/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.3656 - auc: 0.9642
Epoch 11/